In [1]:
import numpy as np
import tensorflow as tf
import sys
from pandas import read_csv
import keras
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from keras.models import model_from_json
import matplotlib.pyplot as plt
from IPython.display import clear_output
np.random.seed(123)  # for reproducibility

# Extra part for memory management
#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#tf.Session(config = config)

config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)

Using TensorFlow backend.


In [2]:
def Test_LSTM(state_condition, structure, peak_hour, pred_hour, LSTM_hour, 
                num_unit, num_epoch, train_index, val_index, test_index, optmzr, loss_ftn):
    #file_condition = '1hPEAK_6hLSTM_log'
    
    len_PRED = pred_hour/peak_hour
    len_LSTM = LSTM_hour/peak_hour
    file_condition = 'SineTest_'+str(peak_hour)+'hPEAK_'+str(pred_hour)+'hPrediction_'+str(LSTM_hour)+'hLSTM'
    
    if len_PRED <= 0:
        print("You entered wrong predction hour or peak hour")
        sys.eixt()
    if len_LSTM <= 0:
        print("You entered wrong LSTM hour or peak hour")
        sys.eixt()
        
    len_PRED = int(len_PRED)
    len_LSTM = int(len_LSTM)
       
    allDataSet = read_csv('C:\\Users\\YKW\\Desktop\\SEP_TEMP\\'+file_condition+'.csv', sep=',')
    allDataSet_size = len(allDataSet)
    
    print("allDataSet: ", allDataSet.shape)
    print("allDataSet_size: ", allDataSet_size)
    print("allDataSet Type: ", type(allDataSet))
    print("allDataSet_size Type: ", type(allDataSet_size))
    
    data_count = np.arange(len_LSTM)+1
    data_count = data_count* (-1)
    data_count = data_count +len_LSTM+1
    data_title = []
    for bi in range(len(data_count)):
        data_title.append('b'+str(int(data_count[bi])))
        
    pred_count = np.arange(len_PRED)
    pred_title = []
    for ai in range(len(pred_count)):
        pred_title.append('a'+str(int(pred_count[ai])))

    #flux = allDataSet[['Year','Mon','Day','Hour','Min','b1','b2','b3','b4','b5','b6']].values
    flux = allDataSet[data_title].values
    pred_flux = allDataSet[pred_title].values
    now_flux = allDataSet['a0'].values
     
    d_flux = np.array([flux])
    d_pred_flux = np.array([pred_flux])
    d_now_flux = np.array([now_flux])
    
    d_flux = np.array(d_flux).reshape(allDataSet_size, len_LSTM,1) ### d_flux = np.array(d_flux).reshape(allDataSet_size, 1, len_LSTM)
#    d_pred_flux = np.array(d_pred_flux).reshape(allDataSet_size, 1, len_PRED)
    d_pred_flux = np.array(d_pred_flux).reshape(allDataSet_size, len_PRED)
    d_now_flux = np.array([d_now_flux]).reshape(allDataSet_size, 1)
    

    def M_LSTM_Stateful_Inspace(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,batch_input_shape=shapes, stateful=True))
        model.add(Dense(len_PRED))
        model.summary()
        return model
            
    def M_LSTM_Stateless_Inspace(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,input_shape=shapes, return_sequences=True, stateful=False))
        model.add(Dropout(0.3))
        model.add(LSTM(num_unit,input_shape=shapes, return_sequences=True, stateful=False))
        model.add(Dropout(0.3))
        model.add(LSTM(num_unit,input_shape=shapes, stateful=False))
        model.add(Dense(len_PRED))
        model.summary()
        return model
    
    def M_LSTM_Stateful_SeaT(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,batch_input_shape=shapes, stateful=True))
        model.add(Dense(7))
        model.add(Dense(len_PRED))
        model.summary()
        return model
            
    def M_LSTM_Stateless_SeaT(shapes):
        model = Sequential()
        model.add(LSTM(num_unit,input_shape=shapes, stateful=False))
        model.add(Dense(7))
        model.add(Dense(len_PRED))
        model.summary()
        return model


    class PlotLosses(keras.callbacks.Callback):
        def on_train_begin(self, logs={}):
            self.i = 0
            self.x = []
            self.losses = []
            self.val_losses = []

            self.fig = plt.figure()

            self.logs = []

        def on_epoch_end(self, epoch, logs={}):

            self.logs.append(logs)
            self.x.append(self.i)
            self.losses.append(logs.get('loss'))
            self.val_losses.append(logs.get('val_loss'))
            self.i += 1
            clear_output(wait=True)
            plt.plot(self.x, self.losses, label="loss")
            plt.plot(self.x, self.val_losses, label="val_loss")
            plt.legend()
            plt.show();
            

    if structure is 'Inspace':
        weightfileName = 'C:\\Users\\YKW\\Desktop\\SEP_TEMP\\ReturnSQ_'+state_condition+'_Shuffled_'+'LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_'+optmzr+'_' + loss_ftn + '_MODEL_WEIGHT'
        testfileName = 'C:\\Users\\YKW\\Desktop\\SEP_TEMP\\ReturnSQ_'+state_condition+'_Shuffled_'+'TEST_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_'+optmzr + '_' + loss_ftn + '_Proton Flux Prediction By LSTM Model Train.csv'
    else:
        if structure is 'SeaT':
            weightfileName = 'C:\\Users\\YKW\\Desktop\\SEP_TEMP\\ReturnSQ_'+state_condition+'_Shuffled_'+'Dense7_LSTM_GOES15_pgt10_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_'+optmzr+'_' + loss_ftn + '_MODEL_WEIGHT'
            testfileName = 'C:\\Users\\YKW\\Desktop\\SEP_TEMP\\ReturnSQ_'+state_condition+'_Shuffled_'+'Dense7_TEST_'+file_condition+'_ep'+str(num_epoch)+'_'+str(num_unit)+'units_'+optmzr + '_' + loss_ftn + '_Proton Flux Prediction By LSTM Model Train.csv'
        else:
            print("You entered wrong ")
            sys.exit()
            

    plot_losses = PlotLosses()
    
    if state_condition is 'Stateful':
        inputFluxShape = (1,1,len_LSTM)
        if structure is 'Inspace':
            model = M_LSTM_Stateful_Inspace(inputFluxShape)
        if structure is 'SeaT':
            model = M_LSTM_Stateful_SeaT(inputFluxShape)
    else:
        if state_condition is 'Stateless':
            inputFluxShape = (len_LSTM,1) ### inputFluxShape = (1,len_LSTM)
            if structure is 'Inspace':
                model = M_LSTM_Stateless_Inspace(inputFluxShape)
            if structure is 'SeaT':
                model = M_LSTM_Stateless_SeaT(inputFluxShape)
        else:
            print("You enterd wrong state_condition!")
            sys.exit()
 

    val_index  = train_index + val_index
    test_index = val_index + test_index
    
    trainX = d_flux[:train_index]
    trainY = d_pred_flux[:train_index]
        
    valX = d_flux[train_index:val_index]
    valY = d_pred_flux[train_index:val_index]
        
    testX = d_flux[val_index:test_index]
    testY = d_pred_flux[val_index:test_index]
    testNowY = d_now_flux[val_index:test_index]

    
    model.compile(loss=loss_ftn, optimizer=optmzr, metrics=['acc'])
     
    if state_condition is 'Stateful':    
        for epoch_idx in range(num_epoch):
            model.fit(trainX, trainY, epochs=1, batch_size=1, verbose=2, shuffle=False, validation_data=(valX, valY))
            model.reset_states()
            print(epoch_idx)
    else:
        if state_condition is 'Stateless': 
            model.fit(trainX, trainY, epochs=num_epoch, batch_size=2520, verbose=2, validation_data=(valX, valY))

        
    model_json = model.to_json()
    with open(weightfileName + ".json", "w") as json_file:
        json_file.write(model_json)
    model.save_weights(weightfileName + ".h5")
    print("Saved model to disk")

    json_file = open(weightfileName + '.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)

    loaded_model.load_weights(weightfileName + ".h5")
    print("Loaded model from disk")
    loaded_model.compile(loss=loss_ftn, optimizer=optmzr, metrics=['acc'])
    

    #date_testY = allDataSet['date'].values[val_index:test_index]
    f = open(testfileName, 'w')
    f.write('Ground')
    for i in range(len_PRED):
        f.write(','+ 'a'+str(i))
    f.write('\n')
        
    for idx2 in range(0, len(testX)):

        labelY = testY[idx2:idx2+1]
        NowY = testNowY[idx2:idx2+1]
        
        pred = []
        pred_in = testX[idx2:idx2+1]
        #pred_in = np.array(pred_in)

        #resultY = loaded_model.predict(pred_in)
        resultY = model.predict(pred_in)
         
        f.write(str(NowY[0][0]))
        for nwr in range(len_PRED):
            f.write(',' + str(resultY[0][nwr]))   
        f.write('\n')
        pred_in = []
        resultY = []
        print(idx2)
        
    f.close()

###################################################################################################

In [3]:
num_epoch = 600
num_unit = 240

Test_LSTM('Stateless', 'Inspace', 1, 24, 48, num_unit, num_epoch, 2520, 360, 720, "Adam",'mean_squared_error')

allDataSet:  (3600, 72)
allDataSet_size:  3600
allDataSet Type:  <class 'pandas.core.frame.DataFrame'>
allDataSet_size Type:  <class 'int'>
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 48, 240)           232320    
_________________________________________________________________
dropout_1 (Dropout)          (None, 48, 240)           0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 48, 240)           461760    
_________________________________________________________________
dropout_2 (Dropout)          (None, 48, 240)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 240)               461760    
_________________________________________________________________
dense_1 (Dense)              (None, 24)                5784      
To

Epoch 81/600
 - 1s - loss: 8.7985e-04 - acc: 0.7782 - val_loss: 1.4268e-04 - val_acc: 0.8833
Epoch 82/600
 - 1s - loss: 9.0522e-04 - acc: 0.7893 - val_loss: 9.1604e-05 - val_acc: 0.8722
Epoch 83/600
 - 1s - loss: 8.2202e-04 - acc: 0.8048 - val_loss: 1.2309e-04 - val_acc: 0.8722
Epoch 84/600
 - 1s - loss: 8.8575e-04 - acc: 0.8075 - val_loss: 7.5203e-05 - val_acc: 0.8833
Epoch 85/600
 - 1s - loss: 8.5146e-04 - acc: 0.8218 - val_loss: 1.2124e-04 - val_acc: 0.9000
Epoch 86/600
 - 1s - loss: 8.5747e-04 - acc: 0.8317 - val_loss: 9.0563e-05 - val_acc: 0.9000
Epoch 87/600
 - 1s - loss: 8.3870e-04 - acc: 0.8341 - val_loss: 7.6211e-05 - val_acc: 0.8833
Epoch 88/600
 - 1s - loss: 8.4617e-04 - acc: 0.8302 - val_loss: 7.0273e-05 - val_acc: 0.8889
Epoch 89/600
 - 1s - loss: 8.2864e-04 - acc: 0.8369 - val_loss: 7.5521e-05 - val_acc: 0.8889
Epoch 90/600
 - 1s - loss: 8.1956e-04 - acc: 0.8306 - val_loss: 7.4341e-05 - val_acc: 0.8833
Epoch 91/600
 - 1s - loss: 7.8960e-04 - acc: 0.8282 - val_loss: 6.3412

Epoch 169/600
 - 1s - loss: 6.1067e-04 - acc: 0.8282 - val_loss: 1.6447e-05 - val_acc: 0.9111
Epoch 170/600
 - 1s - loss: 5.9443e-04 - acc: 0.8294 - val_loss: 4.0518e-05 - val_acc: 0.9167
Epoch 171/600
 - 1s - loss: 6.0878e-04 - acc: 0.8250 - val_loss: 1.3389e-05 - val_acc: 0.9167
Epoch 172/600
 - 1s - loss: 5.9462e-04 - acc: 0.8274 - val_loss: 2.7199e-05 - val_acc: 0.9056
Epoch 173/600
 - 1s - loss: 6.0657e-04 - acc: 0.8226 - val_loss: 2.7371e-05 - val_acc: 0.9111
Epoch 174/600
 - 1s - loss: 6.1662e-04 - acc: 0.8214 - val_loss: 1.8135e-05 - val_acc: 0.9111
Epoch 175/600
 - 1s - loss: 5.5225e-04 - acc: 0.8202 - val_loss: 3.5398e-05 - val_acc: 0.9056
Epoch 176/600
 - 1s - loss: 6.2373e-04 - acc: 0.8218 - val_loss: 1.6742e-05 - val_acc: 0.9222
Epoch 177/600
 - 1s - loss: 6.2231e-04 - acc: 0.8230 - val_loss: 4.9351e-05 - val_acc: 0.9278
Epoch 178/600
 - 1s - loss: 6.2247e-04 - acc: 0.8254 - val_loss: 5.3015e-05 - val_acc: 0.9056
Epoch 179/600
 - 1s - loss: 6.4100e-04 - acc: 0.8147 - val_l

Epoch 257/600
 - 1s - loss: 7.9881e-04 - acc: 0.8345 - val_loss: 4.3625e-04 - val_acc: 0.9111
Epoch 258/600
 - 1s - loss: 8.7185e-04 - acc: 0.8206 - val_loss: 4.4470e-04 - val_acc: 0.9111
Epoch 259/600
 - 1s - loss: 8.9541e-04 - acc: 0.8226 - val_loss: 3.2784e-04 - val_acc: 0.9111
Epoch 260/600
 - 1s - loss: 7.8696e-04 - acc: 0.8270 - val_loss: 1.7561e-04 - val_acc: 0.9111
Epoch 261/600
 - 1s - loss: 6.5193e-04 - acc: 0.8317 - val_loss: 7.1116e-05 - val_acc: 0.9222
Epoch 262/600
 - 1s - loss: 5.4049e-04 - acc: 0.8226 - val_loss: 3.7148e-05 - val_acc: 0.9222
Epoch 263/600
 - 1s - loss: 4.8794e-04 - acc: 0.8425 - val_loss: 3.3113e-05 - val_acc: 0.9278
Epoch 264/600
 - 1s - loss: 5.0753e-04 - acc: 0.8262 - val_loss: 2.8287e-05 - val_acc: 0.9111
Epoch 265/600
 - 1s - loss: 4.7849e-04 - acc: 0.8218 - val_loss: 3.9983e-05 - val_acc: 0.9222
Epoch 266/600
 - 1s - loss: 4.9036e-04 - acc: 0.8313 - val_loss: 2.7639e-05 - val_acc: 0.9056
Epoch 267/600
 - 1s - loss: 4.9011e-04 - acc: 0.8250 - val_l

Epoch 345/600
 - 1s - loss: 4.8218e-04 - acc: 0.8306 - val_loss: 4.9452e-05 - val_acc: 0.9111
Epoch 346/600
 - 1s - loss: 4.3342e-04 - acc: 0.8294 - val_loss: 1.1277e-05 - val_acc: 0.9167
Epoch 347/600
 - 1s - loss: 4.0194e-04 - acc: 0.8306 - val_loss: 1.4061e-05 - val_acc: 0.9167
Epoch 348/600
 - 1s - loss: 4.0604e-04 - acc: 0.8321 - val_loss: 3.5507e-05 - val_acc: 0.9167
Epoch 349/600
 - 1s - loss: 4.2370e-04 - acc: 0.8298 - val_loss: 1.1172e-04 - val_acc: 0.9167
Epoch 350/600
 - 1s - loss: 4.8568e-04 - acc: 0.8361 - val_loss: 1.1862e-04 - val_acc: 0.9056
Epoch 351/600
 - 1s - loss: 5.1548e-04 - acc: 0.8377 - val_loss: 8.4299e-05 - val_acc: 0.9167
Epoch 352/600
 - 1s - loss: 4.7409e-04 - acc: 0.8317 - val_loss: 3.9915e-05 - val_acc: 0.9111
Epoch 353/600
 - 1s - loss: 4.2181e-04 - acc: 0.8381 - val_loss: 2.2115e-05 - val_acc: 0.9167
Epoch 354/600
 - 1s - loss: 4.0068e-04 - acc: 0.8385 - val_loss: 7.6387e-06 - val_acc: 0.9167
Epoch 355/600
 - 1s - loss: 4.0829e-04 - acc: 0.8385 - val_l

Epoch 433/600
 - 1s - loss: 0.0013 - acc: 0.8409 - val_loss: 5.8362e-04 - val_acc: 0.8833
Epoch 434/600
 - 1s - loss: 9.7029e-04 - acc: 0.8278 - val_loss: 3.4314e-04 - val_acc: 0.8944
Epoch 435/600
 - 1s - loss: 6.7809e-04 - acc: 0.8286 - val_loss: 9.7999e-05 - val_acc: 0.8778
Epoch 436/600
 - 1s - loss: 4.2209e-04 - acc: 0.8365 - val_loss: 1.8103e-05 - val_acc: 0.8944
Epoch 437/600
 - 1s - loss: 3.6399e-04 - acc: 0.8337 - val_loss: 1.0291e-04 - val_acc: 0.9000
Epoch 438/600
 - 1s - loss: 4.3475e-04 - acc: 0.8365 - val_loss: 2.7576e-04 - val_acc: 0.8778
Epoch 439/600
 - 1s - loss: 6.2209e-04 - acc: 0.8250 - val_loss: 4.2959e-04 - val_acc: 0.9111
Epoch 440/600
 - 1s - loss: 7.4444e-04 - acc: 0.8444 - val_loss: 3.7665e-04 - val_acc: 0.8778
Epoch 441/600
 - 1s - loss: 7.2883e-04 - acc: 0.8230 - val_loss: 2.3299e-04 - val_acc: 0.8944
Epoch 442/600
 - 1s - loss: 5.7314e-04 - acc: 0.8413 - val_loss: 6.7496e-05 - val_acc: 0.8944
Epoch 443/600
 - 1s - loss: 4.1723e-04 - acc: 0.8302 - val_loss:

Epoch 521/600
 - 1s - loss: 5.5549e-04 - acc: 0.8349 - val_loss: 1.6964e-04 - val_acc: 0.8944
Epoch 522/600
 - 1s - loss: 4.7522e-04 - acc: 0.8349 - val_loss: 1.1576e-04 - val_acc: 0.9222
Epoch 523/600
 - 1s - loss: 4.1492e-04 - acc: 0.8421 - val_loss: 9.7757e-05 - val_acc: 0.9056
Epoch 524/600
 - 1s - loss: 3.8910e-04 - acc: 0.8421 - val_loss: 8.6692e-05 - val_acc: 0.9111
Epoch 525/600
 - 1s - loss: 3.5256e-04 - acc: 0.8377 - val_loss: 4.7197e-05 - val_acc: 0.9000
Epoch 526/600
 - 1s - loss: 3.2534e-04 - acc: 0.8433 - val_loss: 1.7828e-05 - val_acc: 0.9056
Epoch 527/600
 - 1s - loss: 3.0193e-04 - acc: 0.8456 - val_loss: 1.6906e-05 - val_acc: 0.9000
Epoch 528/600
 - 1s - loss: 2.9917e-04 - acc: 0.8421 - val_loss: 1.2554e-05 - val_acc: 0.9056
Epoch 529/600
 - 1s - loss: 2.9552e-04 - acc: 0.8528 - val_loss: 8.4663e-06 - val_acc: 0.9056
Epoch 530/600
 - 1s - loss: 3.0586e-04 - acc: 0.8389 - val_loss: 5.7191e-06 - val_acc: 0.9056
Epoch 531/600
 - 1s - loss: 2.9995e-04 - acc: 0.8369 - val_l

205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
282
283
284
285
286
287
288
289
290
291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327
328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361
362
363
364
365
366
367
368
369
370
371
372
373
374
375
376
377
378
379
380
381
382
383
384
385
386
387
388
389
390
391
392
393
394
395
396
397
398
399
400
401
402
403
404
405
406
407
408
409
410
411
412
413
414
415
416
417
418
419
420
421
422
423
424
425
426
427
428
429
430
431
432
433
434
435
436
437
438
439
440
441
442
443
444
445
446
447
448
449
450
451
452
453
454
